In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Flatten, LeakyReLU, Reshape
from keras.datasets import fashion_mnist
import tensorflow as tf

In [2]:
(x_train, _), (_,_) = fashion_mnist.load_data()
x_train = x_train / 255.0

In [3]:
generator = Sequential()
generator.add(Dense(512, input_shape =[100]))
generator.add(LeakyReLU(alpha=0.2))
generator.add(Dense(256))
generator.add(LeakyReLU(alpha=0.2))
generator.add(Dense(128))
generator.add(LeakyReLU(alpha=0.2))
generator.add(Dense(784))
generator.add(Reshape([28,28,1]))

/opt/anaconda3/envs/multiplabel/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-09-30 13:49:58.592527: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-09-30 13:49:58.592547: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-09-30 13:49:58.592561: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-09-30 13:49:58.592576: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-30 13:49:58.592592: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Crea

In [4]:
discriminator = Sequential()
discriminator.add(Dense(1, input_shape=[28,28,1]))
discriminator.add(Flatten())
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(128))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(64))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(1, activation='softmax'))

In [ ]:
GAN = Sequential([generator,discriminator])
discriminator.compile(optimizer='adam', loss='binary_crossentropy')
discriminator.trainable= False
GAN.compile(optimizer='adam', loss='binary_crossentropy')

In [6]:
epochs = 100
batch_size = 64
noise_shape = 100

for epoch in range(epochs):
    for i in range(x_train.shape[0]//batch_size):
        noise=np.random.normal(size=[batch_size, noise_shape])
        gen_image = generator.predict_on_batch(noise)
        train_dataset = x_train[i*batch_size:(i+1)*batch_size]

        train_label=np.ones(shape=(batch_size,1))
        discriminator.trainable = True
        d_loss_real = discriminator.train_on_batch(train_dataset, train_label)

        train_label = np.zeros(shape=(batch_size,1))
        d_loss_fake = discriminator.train_on_batch(gen_image,train_label)

        noise=np.random.normal(size=[batch_size,noise_shape])
        train_label=np.ones(shape=(batch_size,1))
        discriminator.trainable = False
        d_g_loss_batch = GAN.train_on_batch(noise, train_label)

2024-09-30 13:49:59.005931: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
/opt/anaconda3/envs/multiplabel/lib/python3.9/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (128, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


In [ ]:
noise = np.random.normal(size=[10, noise_shape])

gen_image = generator.predict(noise)
plt.imshow(noise)

fig,axe=plt.subplots(2,5)
idx=0
for i in range(2):
    for j in range(5):
        axe[i,j].imshow(gen_image[idx].reshape(28,28), cmap='gray')
        idx+=1